In [34]:
import os
import pandas as pd
import numpy as np
import math

In [2]:
#data_path = '/home/db600/phd/data/depmap/'
data_path = 'C:\\Users\\dan\\Documents\\phd\\data\\'
#os.getcwd()
os.listdir(data_path)

['alphamissense',
 'biomart',
 'cell_gofs.csv',
 'cell_lofs.csv',
 'dependant',
 'depmap',
 'missing_gene_names.csv',
 'pathway_commons',
 'reactome',
 'reactome_rows_to_drop.csv',
 'string']

In [3]:
dep_data = pd.read_csv(data_path + 'depmap\\CRISPRGeneDependency.csv') 
exp_data = pd.read_csv(data_path + 'depmap\\OmicsExpressionProteinCodingGenesTPMLogp1.csv')
mut_data = pd.read_csv(data_path + 'depmap\\OmicsSomaticMutations.csv', low_memory=False)
model_data = pd.read_csv(data_path + 'depmap\\Model.csv')
ppi_data = pd.read_csv(data_path + 'dependant\\processed_ppi\\pathway_commons_reactome_sif.csv')

In [4]:
dep_data.head()

,ModelID,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZWILCH (55055),ZWINT (11130),ZXDA (7789),ZXDB (158586),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009)
0,ACH-000001,0.055321,0.014039,0.014084,0.033828,0.049511,0.004955,0.026993,0.131006,0.003102,...,0.047757,0.047806,0.007155,0.002537,0.003660,0.154849,0.015792,0.015999,0.006200,0.436777
1,ACH-000004,0.023418,0.048724,0.058084,0.019483,0.049793,0.064472,0.001775,0.071289,0.003732,...,0.044517,0.471009,0.005096,0.015149,0.009804,0.015507,0.319598,0.007778,0.004392,0.048136
2,ACH-000005,0.059552,0.025478,0.009989,0.008775,0.099322,0.099572,0.007544,0.049601,0.047555,...,0.073457,0.281892,0.044789,0.058229,0.029638,0.097357,0.066269,0.025365,0.042530,0.096150
3,ACH-000007,0.023880,0.035082,0.006556,0.004322,0.022200,0.017121,0.009605,0.149378,0.052873,...,0.160608,0.593581,0.017412,0.001892,0.006432,0.029261,0.369101,0.015412,0.165622,0.366325
4,ACH-000009,0.027652,0.074860,0.011021,0.009153,0.021632,0.067758,0.013559,0.216796,0.013660,...,0.181632,0.342313,0.042414,0.002859,0.058070,0.053355,0.651365,0.012016,0.023990,0.312202


In [5]:
mut_data.head()

,Chrom,Pos,Ref,Alt,AF,RefCount,AltCount,GT,PS,VariantType,...,RevelScore,Funseq2Score,PharmgkbID,DidaID,DidaName,GwasDisease,GwasPmID,GTexGene,ModelID,EntrezGeneID
0,chr1,1242864,GC,CT,0.310,19,8,0/1,NaN,DNP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ACH-000839,388581.0
1,chr1,10647969,A,G,0.400,29,19,0|1,10647969.0,SNP,...,0.234,3.0,NaN,NaN,NaN,NaN,NaN,NaN,ACH-000839,54897.0
2,chr1,10648097,T,G,0.349,21,10,0/1,NaN,SNP,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,ACH-000839,54897.0
3,chr1,13198424,G,A,0.833,0,4,0/1,NaN,SNP,...,0.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ACH-000839,400736.0
4,chr1,13225068,A,G,0.396,34,24,0/1,NaN,SNP,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,ACH-000839,391003.0


In [6]:
exp_data.head()

,Unnamed: 0,TSPAN6 (7105),TNMD (64102),DPM1 (8813),SCYL3 (57147),C1orf112 (55732),FGR (2268),CFH (3075),FUCA2 (2519),GCLC (2729),...,H3C2 (8358),H3C3 (8352),AC098582.1 (8916),DUS4L-BCAP29 (115253422),C8orf44-SGK3 (100533105),ELOA3B (728929),NPBWR1 (2831),ELOA3D (100506888),ELOA3 (162699),CDR1 (1038)
0,ACH-001113,4.331992,0.000000,7.364660,2.792855,4.471187,0.028569,1.226509,3.044394,6.500005,...,2.689299,0.189034,0.201634,2.130931,0.555816,0.0,0.275007,0.0,0.0,0.000000
1,ACH-001289,4.567424,0.584963,7.106641,2.543496,3.504620,0.000000,0.189034,3.813525,4.221877,...,1.286881,1.049631,0.321928,1.464668,0.632268,0.0,0.014355,0.0,0.0,0.000000
2,ACH-001339,3.150560,0.000000,7.379118,2.333424,4.228049,0.056584,1.310340,6.687201,3.682573,...,0.594549,1.097611,0.831877,2.946731,0.475085,0.0,0.084064,0.0,0.0,0.042644
3,ACH-001538,5.085340,0.000000,7.154211,2.545968,3.084064,0.000000,5.868390,6.165309,4.489928,...,0.214125,0.632268,0.298658,1.641546,0.443607,0.0,0.028569,0.0,0.0,0.000000
4,ACH-000242,6.729417,0.000000,6.537917,2.456806,3.867896,0.799087,7.208478,5.570159,7.127117,...,1.117695,2.358959,0.084064,1.910733,0.000000,0.0,0.464668,0.0,0.0,0.000000


In [7]:
print(len(ppi_data))
ppi_data.head()

329941


,PARTICIPANT_A,PARTICIPANT_B
0,A1CF,APOBEC2
1,A1CF,APOBEC3A
2,A1CF,APOBEC3B
3,A1CF,APOBEC3C
4,A1CF,APOBEC3H


In [8]:
# Get list of training cell lines used in original program
# Manually saved to CSV earlier
original_cell_lines = pd.read_csv(data_path + 'dependant\\original_training_cell_lines.csv')
original_cell_lines = original_cell_lines.rename(columns={'cell_line': 'CCLEName'})

# Add depmap model ID to the list of original training cell lines
original_cell_lines = pd.merge(original_cell_lines, model_data[['ModelID', 'CCLEName']], on='CCLEName', how='left')

# View the cell line list
original_cell_lines

,CCLEName,ModelID
0,HCC1428_BREAST,ACH-000352
1,HCC1806_BREAST,ACH-000624
2,HCC1937_BREAST,ACH-000223
3,MDAMB231_BREAST,ACH-000768
4,HCC202_BREAST,ACH-000725
5,CAL51_BREAST,ACH-000856
6,MDAMB468_BREAST,ACH-000849
7,KPL1_BREAST,ACH-000028
8,MDAMB415_BREAST,ACH-000876
9,MDAMB157_BREAST,ACH-000621


In [9]:
# Get list of cell lines that present are in:
# 1) Our starting pool of cell lines of interest
# 2) Also present in the dependency, mutation and expression data

cell_lines_of_interest = set(original_cell_lines['ModelID'])
dep_cell_lines = set(dep_data['ModelID'])
exp_cell_lines = set(exp_data['Unnamed: 0'])
mut_cell_lines = set(mut_data['ModelID'])

# Find the intersection of the 4 sets
valid_cell_lines = cell_lines_of_interest & dep_cell_lines & exp_cell_lines & mut_cell_lines

# Use this to select a subset of the original cell lines df
valid_cell_lines = original_cell_lines[original_cell_lines['ModelID'].isin(valid_cell_lines)]

valid_cell_lines

,CCLEName,ModelID
0,HCC1428_BREAST,ACH-000352
1,HCC1806_BREAST,ACH-000624
2,HCC1937_BREAST,ACH-000223
3,MDAMB231_BREAST,ACH-000768
4,HCC202_BREAST,ACH-000725
5,CAL51_BREAST,ACH-000856
6,MDAMB468_BREAST,ACH-000849
7,KPL1_BREAST,ACH-000028
8,MDAMB415_BREAST,ACH-000876
9,MDAMB157_BREAST,ACH-000621


In [10]:
# Get list of genes that present are in:
# 1) The selected PPI data
# 2) Also present in the dependency and expression data 
# (should genes also be present in mutation data? No - if they are not in the muts file, it just means no mutations were found in those genes)

ppi_genes = set(list(ppi_data['PARTICIPANT_A']) + list(ppi_data['PARTICIPANT_B']))

dep_genes = dep_data.columns.map(lambda x: x.split(' ')[0])
dep_genes = set(dep_genes[1:]) # remove item corresponding to the first column

exp_genes = exp_data.columns.map(lambda x: x.split(' ')[0])
exp_genes = set(exp_genes[1:]) # remove item corresponding to the first column

# mut_genes = set(mut_data['HugoSymbol'].unique())

valid_genes = ppi_genes & dep_genes & exp_genes

print(len(valid_genes))
valid_genes

9723


{'TIMP1',
 'CHST2',
 'FBXO27',
 'DOCK3',
 'LUZP4',
 'AGL',
 'UBE2S',
 'ZNF554',
 'MTHFD2',
 'RAB38',
 'HOMER1',
 'CPB2',
 'MMACHC',
 'SEC13',
 'LCN1',
 'HERC5',
 'OR52N1',
 'KCNN4',
 'NCK1',
 'LPAR4',
 'OR5M3',
 'PLA2G2A',
 'JADE2',
 'PAXIP1',
 'CRADD',
 'HAND2',
 'OR10V1',
 'HSP90AB1',
 'GABRA4',
 'TOP1',
 'WNT2',
 'PSMD3',
 'SLC5A10',
 'CHST6',
 'DCLRE1B',
 'ADORA2A',
 'SBF2',
 'KLK12',
 'HSPG2',
 'SV2B',
 'UAP1',
 'SCARF1',
 'SCARA5',
 'MFAP1',
 'AREL1',
 'FGF7',
 'PRMT6',
 'GLB1',
 'LCE3E',
 'AK5',
 'RBP2',
 'FGFBP1',
 'TRPV2',
 'STK4',
 'XRCC3',
 'SLC25A26',
 'RAB40A',
 'HHIP',
 'ALPI',
 'GNAS',
 'PCBP4',
 'TTR',
 'ABCC6',
 'CYP2A7',
 'SIRPA',
 'SLC34A1',
 'KISS1',
 'MIP',
 'FGFBP2',
 'SLC13A4',
 'RND1',
 'OR7A10',
 'CLTC',
 'SCUBE2',
 'USP47',
 'SLC4A8',
 'ABCD4',
 'ARPC2',
 'MPL',
 'GLP2R',
 'ZNF569',
 'ENGASE',
 'POLR2I',
 'MFAP3',
 'FRS3',
 'CLUAP1',
 'TRIM21',
 'DNAJC11',
 'LCT',
 'GUCY1A1',
 'MYL10',
 'ANO9',
 'ELF2',
 'GRIA4',
 'LEF1',
 'CADM1',
 'PHYKPL',
 'FCHO1',
 'OR13J

In [44]:
len(ppi_genes)

12323

In [11]:
# These need to be lists to be used for col headers and row indexes
valid_genes_list = list(valid_genes)
valid_cell_lines_list = list(valid_cell_lines['ModelID'])

# Build a one matrix with the dimensions of the lists above 
one_matrix = np.ones((len(valid_genes_list), len(valid_cell_lines_list)))

# Troubleshooting
print("Length of valid_genes:", len(valid_genes_list))
print("Length of valid_cell_lines:", len(valid_cell_lines_list))
print("Shape of the one matrix:", one_matrix.shape)
print("Type of valid_genes:", type(valid_genes_list))
print("Type of valid_cell_lines:", type(valid_cell_lines_list))

Length of valid_genes: 9723
Length of valid_cell_lines: 37
Shape of the one matrix: (9723, 37)
Type of valid_genes: <class 'list'>
Type of valid_cell_lines: <class 'list'>


In [12]:
base_weights = pd.DataFrame(one_matrix, index=valid_genes_list, columns=valid_cell_lines_list)

In [13]:
base_weights

,ACH-000352,ACH-000624,ACH-000223,ACH-000768,ACH-000725,ACH-000856,ACH-000849,ACH-000028,ACH-000876,ACH-000621,...,ACH-000262,ACH-000060,ACH-000178,ACH-000222,ACH-000281,ACH-000265,ACH-000601,ACH-000138,ACH-000320,ACH-000118
TIMP1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
CHST2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
FBXO27,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
DOCK3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
LUZP4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TRIM24,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
POFUT2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
KRT33B,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
ATM,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [43]:
base_weights.to_csv(data_path + 'dependant\\processed_weights\\reactome_base_weights_original_lines.csv')

In [15]:
# Get subset of mutation data that applies to our valid genes and cell lines only
mutation_df = mut_data[mut_data['ModelID'].isin(valid_cell_lines_list)]
mutation_df = mutation_df[mut_data['HugoSymbol'].isin(valid_genes_list)]

mutation_df

C:\Users\dan\AppData\Local\Temp\ipykernel_16772\2966003349.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  mutation_df = mutation_df[mut_data['HugoSymbol'].isin(valid_genes_list)]


,Chrom,Pos,Ref,Alt,AF,RefCount,AltCount,GT,PS,VariantType,...,RevelScore,Funseq2Score,PharmgkbID,DidaID,DidaName,GwasDisease,GwasPmID,GTexGene,ModelID,EntrezGeneID
29669,chr1,951180,G,A,0.390,26,15,0/1,NaN,SNP,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,ACH-000856,26155.0
29671,chr1,1046671,C,T,0.250,30,8,0/1,NaN,SNP,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,ACH-000856,375790.0
29673,chr1,2412394,C,T,0.391,14,8,0/1,NaN,SNP,...,0.416,3.0,NaN,NaN,NaN,NaN,NaN,NaN,ACH-000856,5192.0
29674,chr1,2512914,G,T,0.488,21,19,0/1,NaN,SNP,...,0.436,1.0,NaN,NaN,NaN,NaN,NaN,NaN,ACH-000856,55229.0
29675,chr1,2561696,C,T,0.320,24,12,0/1,NaN,SNP,...,0.071,2.0,NaN,NaN,NaN,NaN,NaN,NaN,ACH-000856,8764.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296624,chr22,42126611,C,G,0.984,0,72,1|1,NaN,SNP,...,NaN,NaN,PA166156076,NaN,NaN,NaN,NaN,NaN,ACH-000234,1565.0
1296625,chr22,42129130,C,G,0.968,1,71,1|1,NaN,SNP,...,NaN,NaN,PA166156061,NaN,NaN,NaN,NaN,NaN,ACH-000234,1565.0
1296627,chrX,41136962,A,T,0.963,1,57,1|1,NaN,SNP,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,ACH-000234,8239.0
1296628,chrX,51896717,G,A,0.962,1,54,1|1,NaN,SNP,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,ACH-000234,9500.0


In [16]:
# First we separate out the badly pathogenic mutations - these are assumed to result in loss of function
pathogenic = ('FRAME_SHIFT_DEL', 'FRAME_SHIFT_INS', 'NONSENSE', 'NONSTOP', 'START_CODON_INS')

##### NOTE ######
# These are the variations that were considered pathogenic on the original version
# pathogenic = ['Frame_Shift_Del', 'Frame_Shift_Ins','Nonsense_Mutation','Nonstop_Mutation','Stop_Codon_Del']

# I will keep this the same for now (note no 'Stop_Codon_Del' class in the new data - presumably because it's the same a NONSTOP) but note:

# IN_FRAME_DEL - this is likely damaging but could be LOF or GOF -  select which depending on whether its onc, tsg or kinase?
# IN_FRAME_INS - as above
# START_CODON_INS - this is likely to prevent the translation of the protein, so LOF?
# START_CODON_SNP - this may prevent translation if the SNP switched the codon from methianine to another amino acid

# Filter mut_df to only include rows where the variant classification is in the pathogenic list defined above
pathogenic_mutations = mutation_df.loc[mutation_df['VariantInfo'].isin(pathogenic)]

print(len(pathogenic_mutations))
pathogenic_mutations.head()

691


,Chrom,Pos,Ref,Alt,AF,RefCount,AltCount,GT,PS,VariantType,...,RevelScore,Funseq2Score,PharmgkbID,DidaID,DidaName,GwasDisease,GwasPmID,GTexGene,ModelID,EntrezGeneID
29678,chr1,6197724,CT,C,0.534,13,15,0/1,NaN,DEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ACH-000856,6146.0
29699,chr1,20633885,GA,G,0.462,13,11,0/1,NaN,DEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ACH-000856,65018.0
29704,chr1,26773456,C,T,0.389,21,13,0/1,NaN,SNP,...,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,ACH-000856,8289.0
29705,chr1,26779439,TG,T,0.590,15,23,0/1,NaN,DEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ACH-000856,8289.0
29709,chr1,38019345,GA,G,0.448,17,13,0/1,NaN,DEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ACH-000856,51118.0


In [17]:
# Start a lof_mutations df with these very pathogenic mutations 
lof_mutations = pathogenic_mutations[['ModelID', 'HugoSymbol']]
lof_mutations

,ModelID,HugoSymbol
29678,ACH-000856,RPL22
29699,ACH-000856,PINK1
29704,ACH-000856,ARID1A
29705,ACH-000856,ARID1A
29709,ACH-000856,UTP11
...,...,...
1296363,ACH-000234,GPAT3
1296376,ACH-000234,JADE2
1296383,ACH-000234,RUFY1
1296548,ACH-000234,ANPEP


In [18]:
# Open the results from the missense mutation analysis
missense_predictions = pd.read_csv(data_path + 'dependant\\processed_mutations\\alphamissense_pathogenic_predictions.csv')
missense_predictions

,Chrom,Pos,Ref,Alt,AF,RefCount,AltCount,GT,PS,VariantType,...,DidaName,GwasDisease,GwasPmID,GTexGene,ModelID,EntrezGeneID,am_pathogenicity,am_class,am_source,lof_gof
0,chr1,963249,G,A,0.303,22,9,0/1,NaN,SNP,...,NaN,NaN,NaN,NaN,ACH-000856,339451.0,0.9941,likely_pathogenic,primary,lof
1,chr1,2512914,G,T,0.488,21,19,0/1,NaN,SNP,...,NaN,NaN,NaN,NaN,ACH-000856,55229.0,0.9989,likely_pathogenic,primary,lof
2,chr1,6588134,G,A,0.457,19,16,0/1,NaN,SNP,...,NaN,NaN,NaN,NaN,ACH-000856,3104.0,0.8938,likely_pathogenic,primary,lof
3,chr1,11715133,C,A,0.500,20,19,0/1,NaN,SNP,...,NaN,NaN,NaN,NaN,ACH-000856,374946.0,0.5953,likely_pathogenic,primary,lof
4,chr1,13225429,A,G,0.579,15,21,0|1,13225408.0,SNP,...,NaN,NaN,NaN,NaN,ACH-000856,391003.0,0.5949,likely_pathogenic,primary,lof
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2389,chr14,67352948,C,T,0.966,7,245,1|1,NaN,SNP,...,NaN,NaN,NaN,NaN,ACH-000234,51382.0,0.7115,likely_pathogenic,isoform,lof
2390,chr15,59084193,C,T,0.190,129,31,0/1,NaN,SNP,...,NaN,NaN,NaN,NaN,ACH-000234,54778.0,0.9261,likely_pathogenic,isoform,lof
2391,chr15,67187484,G,A,0.560,67,86,0/1,NaN,SNP,...,NaN,NaN,NaN,NaN,ACH-000234,4088.0,0.7934,likely_pathogenic,isoform,lof
2392,chr16,66909296,C,T,0.677,23,52,0/1,NaN,SNP,...,NaN,NaN,NaN,NaN,ACH-000234,1014.0,0.7968,likely_pathogenic,isoform,lof


In [19]:
# Filter for valid genes and cell lines only
missense_predictions = missense_predictions[missense_predictions['ModelID'].isin(valid_cell_lines_list)]
missense_predictions = missense_predictions[missense_predictions['HugoSymbol'].isin(valid_genes_list)]

In [20]:
missense_predictions['lof_gof'].value_counts()

lof    1283
gof      89
Name: lof_gof, dtype: int64

In [21]:
missense_predictions_lof = missense_predictions[missense_predictions['lof_gof']=='lof'][['ModelID', 'HugoSymbol']]
missense_predictions_gof = missense_predictions[missense_predictions['lof_gof']=='gof'][['ModelID', 'HugoSymbol']]

In [22]:
missense_predictions_lof

,ModelID,HugoSymbol
1,ACH-000856,PANK4
5,ACH-000856,COL16A1
6,ACH-000856,CPT2
8,ACH-000856,COL24A1
9,ACH-000856,KCND3
...,...,...
2387,ACH-000234,CACNA1C
2388,ACH-000234,DOCK9
2389,ACH-000234,ATP6V1D
2390,ACH-000234,RNF111


In [23]:
missense_predictions_gof

,ModelID,HugoSymbol
7,ACH-000856,PRKAA2
24,ACH-000856,PIK3CA
121,ACH-000601,RPS6KA6
159,ACH-000725,RAF1
161,ACH-000725,PIK3CA
...,...,...
2147,ACH-000262,RAC1
2156,ACH-000876,PAX3
2318,ACH-000118,ILK
2329,ACH-000118,GSK3A


In [24]:
lof_mutations = pd.concat([lof_mutations, missense_predictions_lof])
lof_mutations = lof_mutations.drop_duplicates()
print(len(lof_mutations))
lof_mutations

1929


,ModelID,HugoSymbol
29678,ACH-000856,RPL22
29699,ACH-000856,PINK1
29704,ACH-000856,ARID1A
29709,ACH-000856,UTP11
29723,ACH-000856,JAK1
...,...,...
2387,ACH-000234,CACNA1C
2388,ACH-000234,DOCK9
2389,ACH-000234,ATP6V1D
2390,ACH-000234,RNF111


In [25]:
gof_mutations = missense_predictions_gof
gof_mutations = gof_mutations.drop_duplicates()
print(len(gof_mutations ))
gof_mutations

87


,ModelID,HugoSymbol
7,ACH-000856,PRKAA2
24,ACH-000856,PIK3CA
121,ACH-000601,RPS6KA6
159,ACH-000725,RAF1
161,ACH-000725,PIK3CA
...,...,...
2147,ACH-000262,RAC1
2156,ACH-000876,PAX3
2318,ACH-000118,ILK
2329,ACH-000118,GSK3A


In [26]:
# Loading the base weights ones matrix
# This can be loaded from the file, or you can use the base_weights dataframe created earlier
#lof_gof_matrix = pd.read_csv(data_path + 'dependant/processed_weights/base_weights_reactome_original_lines.csv', index_col=0)
gof_matrix = base_weights

# Updating the base weights matrix based on the gof_mutations file
# Set matrix values to 10 where they correspond to rows in gof_mutations
for index, row in gof_mutations.iterrows():
    model_id = row['ModelID']
    hugo_symbol = row['HugoSymbol']
    if model_id in gof_matrix.columns and hugo_symbol in gof_matrix.index:
        gof_matrix.at[hugo_symbol, model_id] = 10

gof_matrix# Displaying the first few rows of the updated matrix

,ACH-000352,ACH-000624,ACH-000223,ACH-000768,ACH-000725,ACH-000856,ACH-000849,ACH-000028,ACH-000876,ACH-000621,...,ACH-000262,ACH-000060,ACH-000178,ACH-000222,ACH-000281,ACH-000265,ACH-000601,ACH-000138,ACH-000320,ACH-000118
TIMP1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
CHST2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
FBXO27,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
DOCK3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
LUZP4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TRIM24,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
POFUT2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
KRT33B,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
ATM,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [27]:
# Check how many tens and ones in the matrix. Should match the number of lofs and gofs

# Load your matrix DataFrame
# For example: your_matrix = pd.read_csv('path_to_your_matrix.csv')
# Replace 'your_matrix' with your actual DataFrame variable

# Counting the number of values in the matrix that are equal to 10 and 0
num_values_equal_to_10 = (gof_matrix == 10).sum().sum()
num_values_equal_to_0 = (gof_matrix == 0).sum().sum()

print("Number of values equal to 10:", num_values_equal_to_10)
print("Number of values equal to 0:", num_values_equal_to_0)

Number of values equal to 10: 87
Number of values equal to 0: 0


# Process expression data matrix

In [28]:
# Load your matrix. Assuming it's already loaded into a DataFrame named 'matrix'
# If it needs to be loaded from a file: matrix = pd.read_csv('your_matrix_file.csv')
 
# Transpose the DataFrame
exp_transposed = exp_data.T

# Set the first row as the new headers
exp_transposed.columns = exp_transposed.iloc[0]

# Drop the first row
exp_transposed = exp_transposed[1:]

# Display the first few rows to check
exp_transposed.head()

# Get rid of everything after the space in the index (format is SYMBOL (Entrez ID))
exp_transposed.index = exp_transposed.index.map(lambda x: x.split(' ')[0])

# Remove the name of the index
exp_transposed.index.name = None

print(exp_transposed.index.name)

# Filter matrix2 to have the same rows and columns as matrix1
common_rows = gof_matrix.index.intersection(exp_transposed.index)
common_columns = gof_matrix.columns.intersection(exp_transposed.columns)

filtered_matrix = exp_transposed.loc[common_rows, common_columns]

# Reorder rows and columns in filtered_matrix2 to match matrix1
expression_matrix = filtered_matrix.reindex(index=gof_matrix.index, columns=gof_matrix.columns)

expression_matrix

None


,ACH-000352,ACH-000624,ACH-000223,ACH-000768,ACH-000725,ACH-000856,ACH-000849,ACH-000028,ACH-000876,ACH-000621,...,ACH-000262,ACH-000060,ACH-000178,ACH-000222,ACH-000281,ACH-000265,ACH-000601,ACH-000138,ACH-000320,ACH-000118
TIMP1,6.20594,5.202418,7.034194,9.178715,3.849999,4.931683,5.463034,7.698288,6.650046,9.230117,...,7.008204,7.026579,8.383661,7.650693,7.743892,7.777354,9.250488,8.288359,11.755639,6.757023
CHST2,0.042644,0.056584,0.042644,2.881665,0.028569,3.473787,0.070389,0.042644,0.028569,1.879706,...,1.807355,0.214125,0.014355,2.166715,0.042644,1.405992,0.978196,0.422233,0.632268,0.014355
FBXO27,0.014355,3.533563,4.595146,3.713696,0.201634,3.401903,1.356144,3.243364,1.794936,3.543496,...,2.606442,1.400538,2.817623,0.028569,1.807355,4.140779,1.636915,1.989139,3.800123,0.722466
DOCK3,0.275007,0.070389,0.978196,0.495695,1.22033,0.163499,1.85599,0.111031,0.070389,0.722466,...,0.773996,0.389567,0.097611,0.475085,1.356144,1.589763,0.124328,0.097611,0.189034,0.333424
LUZP4,0.0,0.0,0.042644,0.0,0.0,0.028569,0.0,0.0,0.014355,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TRIM24,4.31034,1.9855,3.058316,3.936402,4.892391,3.940167,4.053111,3.975447,3.200065,3.917432,...,3.883621,3.15866,2.693766,2.386811,3.435629,3.080658,3.314697,2.720278,2.769772,2.799087
POFUT2,3.95977,3.95977,4.088311,4.723012,5.249445,4.830357,4.173127,4.276497,4.745775,4.772941,...,5.070389,3.125982,4.720826,3.721373,4.190615,4.399171,4.06695,4.753284,3.971773,3.69933
KRT33B,0.0,0.0,0.056584,0.565597,0.014355,0.0,0.0,0.0,0.0,0.014355,...,0.042644,0.014355,0.0,0.0,0.111031,0.0,0.0,0.014355,0.0,0.0
ATM,3.117695,3.682573,3.480265,4.034744,2.684819,4.966707,3.401903,2.664483,2.678072,4.011675,...,4.517276,4.519793,5.047887,4.812498,3.740928,5.123914,3.007196,4.075533,4.981396,3.06695


In [29]:
# Check how many null values in the matrix. Shouldn't be any

# Load your matrix DataFrame
# For example: your_matrix = pd.read_csv('path_to_your_matrix.csv')
# Replace 'your_matrix' with your actual DataFrame variable

# Counting the number of values in the matrix that are equal to 10 and 0
num_null_values = (expression_matrix == np.nan).sum().sum()

print("Number of null values:", num_null_values)

Number of null values: 0


In [30]:
# Check that the expression matrix and lof_gof_matrix have identical rows and columns
# This checks that the columns and row names match, and are in the same order

# Check if row indices match
rows_match = gof_matrix.index.equals(expression_matrix.index)

# Check if column names match
columns_match = gof_matrix.columns.equals(expression_matrix.columns)

if rows_match and columns_match:
    print("Both rows and columns match.")
else:
    print("Rows match:", rows_match)
    print("Columns match:", columns_match)

Both rows and columns match.


In [31]:
weights_matrix = expression_matrix * gof_matrix
weights_matrix

,ACH-000352,ACH-000624,ACH-000223,ACH-000768,ACH-000725,ACH-000856,ACH-000849,ACH-000028,ACH-000876,ACH-000621,...,ACH-000262,ACH-000060,ACH-000178,ACH-000222,ACH-000281,ACH-000265,ACH-000601,ACH-000138,ACH-000320,ACH-000118
TIMP1,6.20594,5.202418,7.034194,9.178715,3.849999,4.931683,5.463034,7.698288,6.650046,9.230117,...,7.008204,7.026579,8.383661,7.650693,7.743892,7.777354,9.250488,8.288359,11.755639,6.757023
CHST2,0.042644,0.056584,0.042644,2.881665,0.028569,3.473787,0.070389,0.042644,0.028569,1.879706,...,1.807355,0.214125,0.014355,2.166715,0.042644,1.405992,0.978196,0.422233,0.632268,0.014355
FBXO27,0.014355,3.533563,4.595146,3.713696,0.201634,3.401903,1.356144,3.243364,1.794936,3.543496,...,2.606442,1.400538,2.817623,0.028569,1.807355,4.140779,1.636915,1.989139,3.800123,0.722466
DOCK3,0.275007,0.070389,0.978196,0.495695,1.22033,0.163499,1.85599,0.111031,0.070389,0.722466,...,0.773996,0.389567,0.097611,0.475085,1.356144,1.589763,0.124328,0.097611,0.189034,0.333424
LUZP4,0.0,0.0,0.042644,0.0,0.0,0.028569,0.0,0.0,0.014355,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TRIM24,4.31034,1.9855,3.058316,3.936402,4.892391,3.940167,4.053111,3.975447,3.200065,3.917432,...,3.883621,3.15866,2.693766,2.386811,3.435629,3.080658,3.314697,2.720278,2.769772,2.799087
POFUT2,3.95977,3.95977,4.088311,4.723012,5.249445,4.830357,4.173127,4.276497,4.745775,4.772941,...,5.070389,3.125982,4.720826,3.721373,4.190615,4.399171,4.06695,4.753284,3.971773,3.69933
KRT33B,0.0,0.0,0.056584,0.565597,0.014355,0.0,0.0,0.0,0.0,0.014355,...,0.042644,0.014355,0.0,0.0,0.111031,0.0,0.0,0.014355,0.0,0.0
ATM,3.117695,3.682573,3.480265,4.034744,2.684819,4.966707,3.401903,2.664483,2.678072,4.011675,...,4.517276,4.519793,5.047887,4.812498,3.740928,5.123914,3.007196,4.075533,4.981396,3.06695


In [32]:
# Check a gain of function mutation to ensure the expression value has been multiplied by 10
# 'RPS6KA6', 'ACH-000601' is GOF
print(expression_matrix.loc['RPS6KA6', 'ACH-000601'])
print(weights_matrix.loc['RPS6KA6', 'ACH-000601'])

0.0285691521967709
0.285691521967709


In [33]:
# Check a loss of function mutation - this should not have changed yet
# PINK1, ACH-000856 is LOF
print(expression_matrix.loc['PINK1', 'ACH-000856'])
print(weights_matrix.loc['PINK1', 'ACH-000856'])

3.5372960670908418
3.5372960670908418


In [38]:
# Define the tanh_parameter
a = 0
tanh_parameter = a/2
scale = 'tanh'

# Define the scale functions in a dictionary
scale_functions = {'tanh':lambda x: 0.5-0.5*(math.tanh(tanh_parameter+math.log(x+1e-10))), 'inverse': lambda x:max(int(1/(x+1e-4)),1)}

# Apply the selected scaling function to the dataframe (currently the 'tanh' function)
expression_matrix_scaled = expression_matrix.applymap(scale_functions[scale])

# Now set expression_matrix_scaled values to 1 where they correspond to rows in lof_mutations
for index, row in lof_mutations.iterrows():
    model_id = row['ModelID']
    hugo_symbol = row['HugoSymbol']
    if model_id in expression_matrix_scaled.columns and hugo_symbol in expression_matrix_scaled.index:
        expression_matrix_scaled.at[hugo_symbol, model_id] = 0

expression_matrix_scaled

,ACH-000352,ACH-000624,ACH-000223,ACH-000768,ACH-000725,ACH-000856,ACH-000849,ACH-000028,ACH-000876,ACH-000621,...,ACH-000262,ACH-000060,ACH-000178,ACH-000222,ACH-000281,ACH-000265,ACH-000601,ACH-000138,ACH-000320,ACH-000118
TIMP1,0.025308,0.035631,0.019810,0.011730,0.063201,0.039492,0.032420,0.016594,0.022113,0.011602,...,0.019954,0.019852,0.014028,0.016797,0.016402,0.016264,0.011551,0.014348,0.007184,0.021433
CHST2,0.998185,0.996809,0.998185,0.107481,0.999184,0.076528,0.995070,0.998185,0.999184,0.220590,...,0.234382,0.956161,0.999794,0.175603,0.998185,0.335930,0.511021,0.848694,0.714407,0.999794
FBXO27,0.999794,0.074151,0.045217,0.067606,0.960932,0.079536,0.352221,0.086810,0.236866,0.073766,...,0.128311,0.337666,0.111869,0.999184,0.234382,0.055108,0.271777,0.201748,0.064763,0.657049
DOCK3,0.929689,0.995070,0.511021,0.802753,0.401735,0.973964,0.224987,0.987822,0.995070,0.657049,...,0.625363,0.868235,0.990562,0.815857,0.352221,0.283499,0.984778,0.990562,0.965499,0.899951
LUZP4,1.000000,1.000000,0.998185,1.000000,1.000000,0.999184,1.000000,1.000000,0.999794,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TRIM24,0.051075,0.202339,0.096588,0.060623,0.040103,0.060515,0.057380,0.059509,0.088965,0.061176,...,0.062179,0.091098,0.121119,0.149324,0.078103,0.095325,0.083422,0.119049,0.115319,0.113188
POFUT2,0.059953,0.059953,0.056452,0.042906,0.035018,0.041098,0.054304,0.051845,0.042513,0.042050,...,0.037441,0.092835,0.042944,0.067346,0.053876,0.049134,0.057012,0.042384,0.059613,0.068097
KRT33B,1.000000,1.000000,0.996809,0.757633,0.999794,1.000000,1.000000,1.000000,1.000000,0.999794,...,0.998185,0.999794,1.000000,1.000000,0.987822,1.000000,1.000000,0.999794,1.000000,1.000000
ATM,0.093283,0.068675,0.076265,0.057873,0.121829,0.038959,0.079536,0.123465,0.122368,0.058502,...,0.046716,0.046667,0.037763,0.041390,0.066691,0.036691,0.099570,0.056786,0.038738,0.096097


In [40]:
# Counting the number of values in the matrix that are equal to 0 - should match the number of LOF mutations
num_values_equal_to_0 = (expression_matrix_scaled == 0).sum().sum()

print("Number of values equal to 0:", num_values_equal_to_0)

Number of values equal to 1: 1929


In [42]:
# Write weights to csv
expression_matrix_scaled.to_csv(data_path + 'dependant\\processed_weights\\reactome_raw_tanh_weights_original_lines_{}.csv'.format(a))